In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

Start with the Nashville zipcode outlines and the Davidson County Boundary, then find the centroid of the county to center the map with.

In [2]:
all_crashes_no_hwys = pd.read_csv('../data/clean/all_crashes_no_hwys.csv')
zipcodes = gpd.read_file('../data/clean/zipcodes.geojson')
county = gpd.read_file('../data/clean/Davidson County Border (GIS).geojson')
county

,area_sq_mi,objectid,shape_area,shape_len,geometry
0,525,4,14641027913.6,686109.334953,"MULTIPOLYGON (((-86.52704 36.13598, -86.52914 ..."
1,0,3,675823.52,3786.70930403,"MULTIPOLYGON (((-86.52315 36.13876, -86.52288 ..."


In [3]:
county = gpd.read_file('../data/clean/Davidson County Border (GIS).geojson')
# Use ID 0 for centroid
center = county.geometry.centroid[0]
map_center = [center.y, center.x]
zipcodes = gpd.read_file('../data/clean/zipcodes.geojson')
all_crashes_no_hwys_geo = all_crashes_no_hwys[['accident_number', 'lat', 'long']]
crashes_no_hwys_geo = gpd.GeoDataFrame(all_crashes_no_hwys_geo, crs= zipcodes.crs, geometry=gpd.points_from_xy(all_crashes_no_hwys_geo.long, all_crashes_no_hwys_geo.lat))

C:\Users\chabi\AppData\Local\Temp\ipykernel_10424\692615409.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = county.geometry.centroid[0]


Create a heat map of all the crashes 

In [4]:
all_crashes_no_hwys = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)

folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.125}).add_to(all_crashes_no_hwys)

crashes = []
for row_index, row_values in crashes_no_hwys_geo.iterrows():
    loc = [row_values['lat'], row_values['long']]
    crashes.append(loc)
crashes

HeatMap(crashes, radius = 15, min_opacity = 0.5, gradient={.5: '#ffc2c2', .75: '#ff7970', 1: '#ff0000'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(all_crashes_no_hwys))
folium.LayerControl().add_to(all_crashes_no_hwys)

all_crashes_no_hwys.save('../maps/all_crashes_no_hwys.html')

Now let's see where there are 311 requests along the corridor.

In [5]:
nash_311 = pd.read_csv('../data/clean/nash_311.csv')

In [6]:
nash_311_geo = nash_311[['case_number', 'latitude', 'longitude']]
nash_311_geo = gpd.GeoDataFrame(nash_311_geo, crs= zipcodes.crs, geometry=gpd.points_from_xy(nash_311_geo.longitude, nash_311_geo.latitude))

In [7]:
nash_311 = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)

folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.125}).add_to(nash_311)

requests_311 = []
for row_index, row_values in nash_311_geo.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    requests_311.append(loc)
requests_311

HeatMap(requests_311, radius = 15, min_opacity = 0.5, gradient={.5: '#98ABEE', .75: '#1D24CA', 1: '#201658'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(nash_311))
folium.LayerControl().add_to(nash_311)

nash_311.save('../maps/nash_311.html')

Hmm, It looks like there are some clusters of 311 requests around intersections, particularly in the 37206 zip code. 

Shifting gears to look at mapped locations of issued building permits.

In [17]:
bldg_permits_geo = gpd.read_file('../data/clean/bldg_permits.csv')

TypeError: 'Map' object is not subscriptable

In [19]:
bldg_permits = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)

folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.125}).add_to(bldg_permits)

permits = []
for row_index, row_values in bldg_permits_geo.iterrows():
    loc = bldg_permits_geo.loc['mapped_location']
    permits.append(loc)
permits

HeatMap(permits, radius = 15, min_opacity = 0.5, gradient={.5: '#98ABEE', .75: '#1D24CA', 1: '#201658'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(bldg_permits))
folium.LayerControl().add_to(bldg_permits)

bldg_permits.save('../maps/bldg_permits.html')

KeyError: 'mapped_location'